In [10]:
import math
import time
import pandas as pd

# Define the function that implements the hybrid algorithm


def blendBF(f, a, b, eps=10**(-14)):
    # Initialize the variables
    n = 0
    a1 = a
    a2 = a
    b1 = b
    b2 = b

    while True:
        # Increment the iteration counter
        n += 1
        # Evaluate the function at the endpoints
        fa = f(a)
        fb = f(b)

        # Compute the midpoint and the false position point
        xB = (a + b) / 2
        fxB = f(xB)

        xF = a - (fa * (b - a)) / (fb - fa)
        fxF = f(xF)

        # Choose the one with the smaller absolute value as the root approximation
        if abs(fxB) < abs(fxF):
            x = xB
            fx = fxB
        else:
            x = xF
            fx = fxF

        # Check if the absolute value of fx is less than or equal to the tolerance
        if abs(fx) <= eps:
            # Return the output
            return x, fx, n, a, b

        # Update the interval by applying the bisection and false position methods
        if fa * fxB < 0:
            b1 = xB
        else:
            a1 = xB

        if fa * fxF < 0:
            b2 = xF
        else:
            a2 = xF

        # Set a to the maximum of a1 and a2 and b to the minimum of b1 and b2
        a = max(a1, a2)
        b = min(b1, b2)


# Read the file and store the equations and the intervals in two lists
functions = []
intervals = []
with open('functions.txt', 'r') as file:
    for line in file:
        equation, a, b = line.split(',')  # Split the line by commas
        # Convert the equation to a lambda function and append to the list
        functions.append(eval(f'lambda x: {equation}'))
        # Convert the bounds to floats and append to the list
        intervals.append([float(a), float(b)])

# Define the number of runs for each function
num_runs = 500

# Create a list of tuples containing the function, the interval, and the average time for the hybrid algorithm
results = []
for i in range(len(functions)):
    f = functions[i]  # Get the function from the list
    # Get the interval from the list and use different variables
    a1, b1 = intervals[i]
    total_time = 0
    for j in range(num_runs):
        start_time = time.time()
        # Use the new variables as arguments
        result, fx, iter, x0, x1 = blendBF(f, a1, b1)
        end_time = time.time()
        run_time = end_time - start_time
        total_time += run_time

    average_time = total_time / num_runs
    # Use the new variables in the list
    results.append((f, [a1, b1], average_time))

# Create a pandas dataframe from the results list
df = pd.DataFrame(results, columns=['Function', 'Interval', 'Avg CPU Time'])

# Add columns for the root, the function value, the iterations, the lower bound, and the upper bound, using the blendBF function
df['Root'] = df.apply(lambda row: blendBF(
    row['Function'], row['Interval'][0], row['Interval'][1])[0], axis=1)
df['f(x)'] = df.apply(lambda row: blendBF(row['Function'],
                                          row['Interval'][0], row['Interval'][1])[1], axis=1)
df['Iter'] = df.apply(lambda row: blendBF(
    row['Function'], row['Interval'][0], row['Interval'][1])[2], axis=1)
# Add the lower bound column
df['a'] = df.apply(lambda row: blendBF(
    row['Function'], row['Interval'][0], row['Interval'][1])[3], axis=1)
# Add the upper bound column
df['b'] = df.apply(lambda row: blendBF(
    row['Function'], row['Interval'][0], row['Interval'][1])[4], axis=1)

# Drop the function and interval columns
df = df.drop(['Function', 'Interval'], axis=1)

# Add the problem column with labels
df['Problem'] = [f'$P{i+1}$' for i in range(len(functions))]

# Reorder the columns
# Include the lower and upper bound columns
df = df[['Problem', 'Iter', 'Avg CPU Time', 'Root', 'f(x)', 'a', 'b']]

# Rename the columns
df = df.rename(columns={'Root': 'Approximate Root',
               'f(x)': 'Function Value', 'a': 'Lower Bound', 'b': 'Upper Bound'})

# Use the scientific notation format for the Avg CPU Time column
pd.options.display.float_format = ' {:.16f}'.format

# Save the dataframe to an excel file with a different name, such as hybrid-table.xlsx
df.to_csv('hybrid-table.csv', index=False)

# Display the dataframe in Jupyter Notebook
df

,Problem,Iter,Avg CPU Time,Approximate Root,Function Value,Lower Bound,Upper Bound
0,$P1$,10,0.0000240931510925,1.3652300134140964,-0.0000000000000071,1.3652300134137789,1.3675001980274413
1,$P2$,1,0.0000019979476929,2.0000000000000000,0.0000000000000000,0.0000000000000000,4.0000000000000000
2,$P3$,10,0.0000151052474976,0.6931471805599453,0.0000000000000000,0.6931471805599334,0.6951627064644150
3,$P4$,6,0.0000079240798950,3.1415926535897931,0.0000000000000001,3.1415903579556947,3.1415926536048882
4,$P5$,1,0.0000040078163147,1.0000000000000000,0.0000000000000000,1.0000000000000000,2.5000000000000000
5,$P6$,1,0.0000020699501038,-2.0000000000000000,0.0000000000000000,-2.5000000000000000,-1.5000000000000000
6,$P7$,8,0.0000159220695496,0.7390851332151606,0.0000000000000001,0.7390851332151470,0.7422270732175922
7,$P8$,1,0.0000020012855530,3.0000000000000000,0.0000000000000000,2.0000000000000000,4.0000000000000000
8,$P9$,1,0.0000020003318787,0.0000000000000000,0.0000000000000000,-1.0000000000000000,1.0000000000000000
9,$P10$,8,0.0000299997329712,3.1117486563092474,0.0000000000000000,3.1085379927858856,3.1117486563092536
